In [65]:
# Useful for defining quantities
from astropy import units as u

# Earth focused modules, ISS example orbit and time span generator
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.twobody import Orbit
from poliastro.spacecraft import Spacecraft
from poliastro.examples import iss
from poliastro.util import time_range
from poliastro.bodies import Earth

In [58]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

In [59]:
radius_E = 6400

In [83]:
Earth.k

<Quantity 3.98600442e+14 m3 / s2>

In [98]:
@interact(h=FloatSlider(min=7000, max=19000, step=10, continuous_update=False),
          i=FloatSlider(min=0, max=90, step=1, continuous_update=False),
          duration=IntSlider(min=10, max=1000, step=10, continuous_update=False),
          max_day_bn_rep=IntSlider(values=2,min=1, max=30, step=1, continuous_update=False))
def groundtract(h, i, duration,max_day_bn_rep):
    # Build spacecraft instance
    a = (radius_E+h) * u.km
    ecc = 0 * u.one
    inc = i * u.deg
    raan = 0 * u.deg
    argp = 0 * u.deg
    nu = 0 * u.deg
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)
    print("Original period: ", orb.period.to(u.h))
    #adjust to integer orbits in a day
    T_E =  Earth.rotational_period.to(u.s)
    ratio =T_E/orb.period 
    ratio = np.floor(max_day_bn_rep*ratio)/float(max_day_bn_rep)*u.one
    new_period = T_E/ratio
    a = (Earth.k*(new_period/2/np.pi)**2)**(1./3)
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

    print("Adjusted period: ", orb.period.to(u.h), "Adjusted altitude: ", (a - radius_E*u.km).to(u.km))
    sat = EarthSatellite(orb, None)
    t_span = time_range(orb.epoch, periods=duration*5, end=orb.epoch+duration * u.h)
    gp = GroundtrackPlotter()
    gp.update_layout(title="Groundtrack")

    # Plot previously defined EarthSatellite object
    gp.plot(
        sat,
        t_span,
        label="Sat",
        color = "red",
        marker={"size": 10, "symbol": "triangle-right", "line": {"width": 1, "color": "black"}},
    )
    gp.fig.show()

interactive(children=(FloatSlider(value=7000.0, continuous_update=False, description='h', max=19000.0, min=700…

In [62]:
orb.period

<Quantity 15437.18746137 s>

In [49]:
gp.fig.show()

In [41]:
# Generate an instance of the plotter, add title and show latlon grid


In [42]:
gp.update_geos(projection_type="orthographic")
gp.fig.show()